In [1]:
import os
import random
from typing import Literal

from dotenv import load_dotenv
from pathlib import Path

env_path = Path('..') / '.env.local'
load_dotenv(dotenv_path=env_path)

url = os.getenv("NEXT_PUBLIC_SUPABASE_URL")
key = os.getenv("NEXT_PUBLIC_SUPABASE_ANON_KEY")
NUMBER_OF_PARTICIPANTS = 12
DRAFT_NUM = 1
NUMBER_OF_PLAYERS = 10
COMPETITION_ID = 6
POOL_ID = 19
MY_ROSTER_ID = 58
CURRENT_TOURNAMENT_ROUND = 2
FINAL_ROUND_IN_TOURNAMENT = 7

In [4]:
from supabase import create_client, Client

In [5]:
supabase: Client = create_client(url, key)


In [8]:
_StrategyType = Literal["RANDOM", "TOURNAMENT_POINTS"]
def generate_autodraft_rankings(pool_id = POOL_ID, draft_num = DRAFT_NUM, strategy: _StrategyType = "RANDOM"):
    competition_id = supabase.table('pool').select('competition_id').eq('pool_id', pool_id).execute().data[0]['competition_id']
    print(f"Competition ID: {competition_id}")
    round_num = supabase.table('poolrule_draft').select('round_num').eq('pool_id', pool_id).eq('draft_num', draft_num).execute().data[0]['round_num']
    print(f"Round {round_num}")
    print(f"Autodraft Strategy: {strategy}")
    if strategy == "TOURNAMENT_POINTS":
        ranked_players = supabase.table('player_total_score_view').select('player_unique, total_points').eq('competition_id', competition_id).order('total_points', desc=True, nullsfirst=False).execute().data
        # print(ranked_players)
        available_players = [i['player_unique'] for i in supabase.table('view_available_players').select('player_unique').eq('pool_id', pool_id).execute().data]
        # rankings = [f"{i['player_unique']}-{i['total_points']}" for i in ranked_players if i['player_unique'] in available_players]
        rankings =  [i['player_unique'] for i in ranked_players if i['player_unique'] in available_players]
        return rankings
    elif strategy == "RANDOM":
        available_players = [i['player_unique'] for i in supabase.table('view_available_players').select('player_unique').eq('pool_id', pool_id).execute().data]
        random.shuffle(available_players)
        print(":::available players", available_players)
        return available_players

In [9]:
print(generate_autodraft_rankings(POOL_ID, DRAFT_NUM, strategy="RANDOM"))

Competition ID: 6
Round 2
Autodraft Strategy: RANDOM
:::available players ['harrison-alexander-1', 'mouhamed-dioubate-1', 'jamari-mcdowell-1', 'steele-venters-1', 'max-triplett-1', 'ian-burns-3', 'aaron-scott-5', 'kasparas-jakucionis-1', 'reyne-smith-1', 'flory-bidunga-1', 'chris-ford-5', 'tony-osburn-1', 'dusty-stromer-2', 'townsend-tripple-2', 'addarin-scott-1', 'marcus-allen-3', 'cham-okey-1', 'jalen-keago-1', 'josh-ozabor-1', 'jadrian-tracey-1', 'mitchell-saxen-1', 'motiejus-krivas-1', 'adou-thiero-1', 'dante-maddoxjr-1', 'grant-stubblefield-1', 'tyler-mcghie-1', 'jack-anderson-4', 'will-tschetter-1', 'tj-caldwell-1', 'brian-waddell-1', 'jaheim-hudson-1', 'howard-eisley-jr-1', 'noah-waterman-1', 'aniwaniwa-tait-jones-1', 'jamari-phillips-1', 'demarion-watson-1', 'sam-walters-3', 'cade-pendleton-1', 'kamryn-thomas-1', 'federiko-federiko-1', 'fedor-zugic-1', 'andrew-mckeever-1', 'dayton-forsythe-1', 'ian-jackson-1', 'lazar-stefanovic-1', 'roddy-gayle-jr-1', 'mustapha-amzil-1', 'jacks

In [23]:
def generate_rankings_dict(pool_id = POOL_ID, draft_num: int = DRAFT_NUM, max_players: int = 1000, num_participants: int = NUMBER_OF_PARTICIPANTS, autodraft_strategy: _StrategyType = "RANDOM"):
    rostrankmap = {}
    draft_order = get_draft_order(draft_num=draft_num, pool_id=pool_id, num_participants=num_participants)
    rankings = supabase.table('draft_view').select('ranking, roster_id, player_unique').eq('pool_id', pool_id).eq('draft_num', draft_num).execute()
    print(rankings)
    print(len(rankings.data))
    for row in rankings.data:
        if row['roster_id'] == MY_ROSTER_ID:
            print(row)

        if 'roster_id' not in row:
            continue
        roster_id = str(row['roster_id'])
        ranking = int(row['ranking'])
        player_unique = row['player_unique']
        if roster_id not in rostrankmap:
            rostrankmap[roster_id] = [None for i in range(0, max_players)]
            rostrankmap[roster_id][ranking] = player_unique
        else:
            try:
              rostrankmap[roster_id][ranking] = player_unique
            except:
                print(":::ERROR:::")
                print(roster_id)
                print(rostrankmap[roster_id])
                print(ranking)
    cleaned_map = {}
    for rost in rostrankmap:
        mapped_rankings = rostrankmap[rost]
        cleaned_map[str(rost)] = [i for i in mapped_rankings if i is not None]
    for rost in draft_order:
        if str(rost) not in cleaned_map:
            cleaned_map[str(rost)] = generate_autodraft_rankings(pool_id, draft_num, autodraft_strategy)
                
    return cleaned_map

In [11]:
print(generate_rankings_dict())

NameError: name 'get_draft_order' is not defined

In [115]:
def get_active_players(current_round: int = CURRENT_TOURNAMENT_ROUND, competition_id: int = COMPETITION_ID):
    available_players = supabase.table('view_active_players').select('player_unique, round_eliminated').eq('competition_id', competition_id).execute()
    return set([row['player_unique'] for row in available_players.data if row['round_eliminated'] is None])

In [116]:
print(get_active_players(current_round=CURRENT_TOURNAMENT_ROUND, competition_id=COMPETITION_ID))

{'stevie-mitchell-1', 'duke-miles-1', 'christian-fermin-1', 'nate-johnson-8', 'joe-charles-2', 'dante-mayo-jr-1', 'jaden-karuletwa-1', 'traivar-jackson-1', 'josh-ibukunoluwa-1', 'raheim-moss-1', 'cham-okey-1', 'marcus-rigsby-jr-1', 'anthony-robinson-ii-1', 'myles-thompson-2', 'rakease-passmore-1', 'joseph-bamisile-1', 'wilder-evers-1', 'kobe-elvis-1', 'trey-green-2', 'jaelyn-withers-1', 'jerome-brewer-jr-1', 'moussa-cisse-1', 'mookie-cook-1', 'nate-heise-1', 'david-fonville-1', 'marvin-musiime-kamali-1', 'kezza-giffa-1', 'sam-king-2', 'khamani-cooper-1', 'kyle-jorgensen-1', 'kayden-fish-1', 'cameron-brown-2', 'zhuric-phelps-1', 'jack-janicki-1', 'viktor-mikic-1', 'jaxon-hartman-1', 'daniel-nauseef-1', 'kevin-gad-1', 'lj-cason-1', 'steven-ashworth-1', 'jeremy-fears-jr-1', 'tafara-gapare-1', 'tobe-awaka-1', 'chase-johnston-1', 'dre-davis-3', 'bennett-stirtz-1', 'terrance-arceneaux-1', 'ian-sabourin-1', 'alex-condon-1', 'tarris-reed-jr-1', 'sven-djopmo-1', 'riley-kugel-1', 'tyreek-smith-1

# Draft

In [12]:
def run_draft(expected_participants: int = NUMBER_OF_PARTICIPANTS, round_start: int = CURRENT_TOURNAMENT_ROUND, round_end: int = FINAL_ROUND_IN_TOURNAMENT, pool_id: int = POOL_ID, draft_num: int = DRAFT_NUM, starting_pick_num: int = 1, autodraft_strategy: _StrategyType = "RANDOM"):
    draft_rules = supabase.table('poolrule_draft').select('*').eq('pool_id', pool_id).eq('draft_num', draft_num).execute()
    print(draft_rules)
    roster_count = draft_rules.data[0]['roster_count']
    round_num = draft_rules.data[0]['round_num']
    draft_time = draft_rules.data[0]['draft_time']
    draft_order = draft_rules.data[0]['draft_order']
    draft_order = get_draft_order(num_participants=expected_participants, pool_id=pool_id, draft_num=draft_num)
    rankings_dict = generate_rankings_dict(pool_id, draft_num, autodraft_strategy=autodraft_strategy)
    available_players_set = get_active_players()
    last_pick_dict = {str(key): 0 for key in draft_order}
    print(last_pick_dict)
    print(draft_order)
    pick_num = starting_pick_num
    draft_insert = []
    # print(rankings_dict)
    print(rankings_dict.keys())
    print(":::RANKINGS DICT:::", rankings_dict)
    print(available_players_set)
    print(last_pick_dict)
    print(":::DRAFT ORDER:::", draft_order)
    print("Starting Draft")
    for draft_round in range (0, roster_count):
        for round_pick in range (0, expected_participants):
            made_pick = False
            current_drafter = str(draft_order[round_pick])
            for pick in range (last_pick_dict[current_drafter], len(rankings_dict[current_drafter])):
                player_unique = rankings_dict[current_drafter][pick]
                if player_unique in available_players_set:
                    print(f'Player {player_unique} picked by {current_drafter} at pick {pick_num}')
                    last_pick_dict[current_drafter] = pick
                    available_players_set.remove(player_unique)
                    pick_row = {
                        'roster_id': int(current_drafter),
                        'player_unique': player_unique,
                        'round_start': round_start,
                        'round_end': round_end,
                        'pick_number': pick_num,
                        'draft_num': draft_num
                    }
                    draft_insert.append(pick_row)
                    made_pick = True
                    break
            if not made_pick:
                raise Exception(f'No pick made by {current_drafter} at pick {pick_num} round {draft_round+1}')
            pick_num += 1
            if round_pick == expected_participants - 1:
                print(f'Round {draft_round} complete')
                draft_order.reverse()
    print(draft_insert)
    return draft_insert
    
def record_draft_in_db(draft_insert):
    supabase.table('roster_player').upsert( draft_insert, ignore_duplicates=True ).execute()

In [25]:
draft_results = run_draft(round_start=CURRENT_TOURNAMENT_ROUND, draft_num=DRAFT_NUM, expected_participants=NUMBER_OF_PARTICIPANTS, starting_pick_num=1)

data=[{'draft_num': 1, 'pool_id': 19, 'roster_count': 10, 'draft_time': '2025-03-19T21:00:00', 'draft_order': 0, 'round_num': 2}] count=None
data=[{'ranking': 1, 'roster_id': 43, 'player_unique': 'cooper-flagg-1'}, {'ranking': 2, 'roster_id': 43, 'player_unique': 'johni-broome-1'}, {'ranking': 4, 'roster_id': 43, 'player_unique': 'kon-knueppel-1'}, {'ranking': 5, 'roster_id': 43, 'player_unique': 'tyrese-proctor-1'}, {'ranking': 6, 'roster_id': 43, 'player_unique': 'walter-claytonjr-1'}, {'ranking': 7, 'roster_id': 43, 'player_unique': 'alijah-martin-1'}, {'ranking': 8, 'roster_id': 43, 'player_unique': 'will-richard-2'}, {'ranking': 9, 'roster_id': 43, 'player_unique': 'chad-baker-mazara-1'}, {'ranking': 10, 'roster_id': 43, 'player_unique': 'lj-cryer-1'}, {'ranking': 11, 'roster_id': 43, 'player_unique': 'rj-luis-1'}, {'ranking': 12, 'roster_id': 43, 'player_unique': 'chaz-lanier-1'}, {'ranking': 13, 'roster_id': 43, 'player_unique': 'pj-haggerty-1'}, {'ranking': 14, 'roster_id': 43,

In [42]:
print([i for i in range(0,12)])

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]


In [26]:
record_draft_in_db(draft_results)

In [16]:
import random
import datetime
import json
def generate_draft_order(pool_id: int = POOL_ID, draft_num: int = DRAFT_NUM):
    rosters = supabase.table('roster_full_view').select('roster_id, rankings_submitted').eq('pool_id', pool_id).execute()
    draft_rules = supabase.table('poolrule_draft').select('*').eq('pool_id', pool_id).eq('draft_num', draft_num).execute()
    draft_rule_order = draft_rules.data[0]['draft_order']
    roster_list = []
    if draft_num == 1 or draft_rules.data[0]['draft_order'] == 0:
        roster_list = [row['roster_id'] for row in rosters.data]
        random.shuffle(roster_list)
    else:
        prev_draft_order = [row['roster_id'] for row in supabase.table('roster_draftorder').select('roster_id').eq('draft_num', draft_num-1).execute().data]
        if draft_rule_order == 1:
            roster_list = prev_draft_order
        elif draft_rule_order == -1:
            prev_draft_order.reverse()
            roster_list = prev_draft_order
        print(roster_list)
    order_insert = []
    created_at = json.dumps(datetime.datetime.now(), indent=4, sort_keys=True, default=str)
    for idx, roster_id in enumerate(roster_list):
        row = {
            'draft_num': draft_num,
            'roster_id': roster_id,
            'created_at': created_at,
            'draft_order': idx+1
        }
        order_insert.append(row)
    supabase.table('roster_draftorder').upsert( order_insert, on_conflict='draft_num, roster_id', ignore_duplicates=False).execute()
    return roster_list

In [53]:
print(generate_draft_order(draft_num=2))

[35, 33, 27, 24, 31, 32, 28, 30, 26, 38, 21, 29, 11, 9, 13, 14, 16, 12, 17, 10, 18]
[35, 33, 27, 24, 31, 32, 28, 30, 26, 38, 21, 29, 11, 9, 13, 14, 16, 12, 17, 10, 18]


In [14]:
def get_draft_order(num_participants: int = NUMBER_OF_PARTICIPANTS, pool_id=POOL_ID, draft_num=DRAFT_NUM):
    draft_order_data = supabase.table('draft_order_view').select('*').eq('pool_id', pool_id).eq('draft_num', draft_num).execute()
    if len(draft_order_data.data) == num_participants:
        draft_order = [None for i in range(0, num_participants + 1)]
        for row in draft_order_data.data:
            draft_order[row['draft_order']] = row['roster_id']
    else:
        draft_order = generate_draft_order(pool_id, draft_num)
    return [i for i in draft_order if i is not None]

In [54]:
print(get_draft_order())

[35, 33, 27, 24, 31, 32, 28, 30, 26, 38, 21, 29]


In [83]:
get_draft_info()

[{'draft_num': 1, 'pool_id': 8, 'roster_count': 12, 'draft_time': '2023-03-16T11:00:00', 'draft_order': 0, 'round_num': 2}]


In [168]:
def add_next_available_player(roster_id: int, draft_num: int, pool_id: int = POOL_ID, 
                              current_round: int = CURRENT_TOURNAMENT_ROUND,
                              final_round: int = 7,
                              autodraft_strategy: str = "RANDOM"):
    """
    Add the next available player from a roster's ranking to the roster.
    
    Args:
        roster_id: The roster ID to add a player to
        draft_num: The draft number
        pool_id: The pool ID
        competition_id: The competition ID
        current_round: The current tournament round
        final_round: The final round of the tournament
        autodraft_strategy: Strategy for auto-drafting ("RANDOM" or "TOURNAMENT_POINTS")
    
    Returns:
        dict: The added player's information or None if no player was added
    """
    
    competition_id = supabase.table('pool').select('competition_id').eq('pool_id', pool_id).execute().data[0]['competition_id']
    
    # Get draft rules to determine target roster size
    draft_rules = supabase.table('poolrule_draft').select('roster_count').eq('pool_id', pool_id).eq('draft_num', draft_num).execute()
    if not draft_rules.data:
        print(f"No draft rules found for draft {draft_num} in pool {pool_id}")
        return None
    
    target_roster_size = draft_rules.data[0]['roster_count']
    
    competition_roster_ids = [r['roster_id'] for r in supabase.table('roster').select('roster_id').eq('pool_id', pool_id).execute().data]
    
    # Get current roster size
    current_roster = supabase.table('roster_player').select('player_unique').eq('roster_id', roster_id).eq('draft_num', draft_num).execute()
    current_roster_size = len(current_roster.data)
    
    # If roster is already at or over the target size, do nothing
    if current_roster_size >= target_roster_size:
        print(f"Roster {roster_id} already has {current_roster_size} players (target: {target_roster_size})")
        return None
    
    eliminated_teams = set([r['team_unique'] for r in supabase.table('team_competition').select('team_unique, round_eliminated, competition_id').eq('competition_id', competition_id).execute().data if r['round_eliminated'] is not None and r['round_eliminated'] < current_round])
    
    drafted_players = set([r['player_unique'] for r in supabase.table('roster_player').select('player_unique').in_('roster_id', competition_roster_ids).execute().data])
    all_players = set([r['player_unique'] for r in supabase.table('player_competition').select('player_unique, round_declared_inactive, team_unique').eq('competition_id', competition_id).execute().data if r['team_unique'] not in eliminated_teams and (r['round_declared_inactive'] is None or r['round_declared_inactive'] >= current_round)])
    
    # Get active players
    available_players_set = all_players - drafted_players
    
    # Get rankings for this roster
    rankings_dict = generate_rankings_dict(pool_id, draft_num, autodraft_strategy=autodraft_strategy)
    if str(roster_id) not in rankings_dict:
        print(f"No rankings found for roster {roster_id}")
        return None
    
    # Find the next available player from the rankings
    roster_rankings = rankings_dict[str(roster_id)]
    next_player = None
    
    for player_unique in roster_rankings:
        if player_unique in available_players_set:
            next_player = player_unique
            break
    
    # If no player found, return None
    if not next_player:
        print(f"No available players found to add to roster {roster_id}")
        return None
    
    # Get the next pick number
    pick_query = supabase.table('roster_player').select('pick_number').eq('draft_num', draft_num).order('pick_number', desc=True).limit(1).execute()
    next_pick = pick_query.data[0]['pick_number'] + 1 if pick_query.data else 1
    
    # Create the player row
    player_row = {
        'roster_id': int(roster_id),
        'player_unique': next_player,
        'round_start': current_round,
        'round_end': final_round,
        'pick_number': next_pick,
        'draft_num': draft_num
    }
    
    # Insert into database
    result = supabase.table('roster_player').insert(player_row).execute()
    print(f"Added player {next_player} to roster {roster_id} as pick {next_pick}")
    return player_row

In [147]:
def fill_rosters_in_draft_order(draft_num: int, pool_id: int = POOL_ID, 
                               competition_id: int = COMPETITION_ID, 
                               current_round: int = CURRENT_TOURNAMENT_ROUND,
                               final_round: int = 7,
                               num_participants: int = 12,
                               autodraft_strategy: str = "RANDOM"):
    """
    Fill rosters in draft order with available players.
    
    Args:
        draft_num: The draft number
        pool_id: The pool ID
        competition_id: The competition ID
        current_round: The current tournament round
        final_round: The final round of the tournament
        num_participants: Number of participants
        autodraft_strategy: Strategy for auto-drafting ("RANDOM" or "TOURNAMENT_POINTS")
    
    Returns:
        int: The number of players added
    """
    # Get draft rules to determine target roster size
    draft_rules = supabase.table('poolrule_draft').select('roster_count').eq('pool_id', pool_id).eq('draft_num', draft_num).execute()
    if not draft_rules.data:
        print(f"No roster size defined for draft {draft_num} in pool {pool_id}")
        return 0
    
    target_roster_size = draft_rules.data[0]['roster_count']
    print(f"Target roster size: {target_roster_size}")
    
    # Get draft order
    draft_order = get_draft_order(num_participants=num_participants, pool_id=pool_id, draft_num=draft_num)
    if not draft_order:
        print(f"No draft order found for draft {draft_num} in pool {pool_id}")
        return 0
    
    added_count = 0
    
    # Loop through the draft order until no more players can be added or all rosters are filled
    changes_made = True
    while changes_made:
        changes_made = False
        for roster_id in draft_order:
            # Get current roster size
            current_roster = supabase.table('roster_player').select('player_unique').eq('roster_id', roster_id).eq('draft_num', draft_num).execute()
            current_roster_size = len(current_roster.data)
            
            if current_roster_size < target_roster_size:
                result = add_next_available_player(
                    roster_id, 
                    draft_num, 
                    pool_id, 
                    current_round,
                    final_round,
                    autodraft_strategy
                )
                if result:
                    added_count += 1
                    changes_made = True
                    # Break after adding one player to maintain draft order rotation
                    break
    
    print(f"Added {added_count} players to rosters for draft {draft_num}")
    return added_count


In [121]:
def drop_inactive_players(draft_num: int, pool_id: int = POOL_ID):
    """
    Drop inactive players from rosters.
    
    This function checks if players on rosters are inactive or were eliminated
    before the draft round, and drops them from the roster if so.
    
    Args:
        draft_num: The draft number
        pool_id: The pool ID
        competition_id: The competition ID
    
    Returns:
        int: The number of players dropped
    """
    # Get the competition_id from the pool_id
    pool_data = supabase.table('pool').select('competition_id').eq('pool_id', pool_id).execute()
    competition_id = pool_data.data[0]['competition_id']
    
    # Get the round that corresponds to this draft
    draft_rule = supabase.table('poolrule_draft').select('round_num').eq('pool_id', pool_id).eq('draft_num', draft_num).execute()
    if not draft_rule.data:
        print(f"No draft rule found for draft {draft_num} in pool {pool_id}")
        return 0
    
    round_num = draft_rule.data[0]['round_num']
    
    # Get the roster ids for this pool
    rosters = supabase.table('roster').select('roster_id').eq('pool_id', pool_id).execute()
    if not rosters.data:
        print(f"No rosters found for pool {pool_id}")
        return 0
    
   # Get all players drafted to these rosters in this draft number
    roster_players = supabase.table('roster_player').select('roster_id, player_unique, draft_num, round_start').in_('roster_id', [val["roster_id"] for val in rosters.data]).eq('draft_num', draft_num).execute()
    if not roster_players.data:
        print(f"No roster players found for draft {draft_num}")
        return 0
    
    # Get player competition data to check status
    player_comp_data = supabase.table('player_competition').select('player_unique, inactive, round_declared_inactive').eq('competition_id', competition_id).execute()
    # inactive_players_at_draft_time = [p['player_unique'] for p in player_comp_data.data if p.get('round_declared_inactive') is not None and p.get('round_declared_inactive') < round_num]
    
    dropped_count = 0
    for player in roster_players.data:
        player_id = player['player_unique']
        roster_id = player['roster_id']
        
        # Find player's competition data
        player_data = next((p for p in player_comp_data.data if p['player_unique'] == player_id), None)
        
        # Check if player is inactive since before this draft
        if player_data and (
            (player_data.get('round_declared_inactive') is not None and player_data.get('round_declared_inactive') < round_num) 
            # or
            # (player_data.get('inactive', False) == True)
        ):
            result = supabase.table('roster_player').delete().eq('roster_id', roster_id).eq('player_unique', player_id).eq('draft_num', draft_num).execute()
            if result.data:
                dropped_count += 1
                inactive_info = f"inactive since round {player_data.get('inactive_since_round')}" if player_data.get('inactive_since_round') else "inactive"
                print(f"Dropped player {player_id} from roster {roster_id} ({inactive_info})")
    
    print(f"Dropped {dropped_count} inactive players from draft {draft_num}")
    return dropped_count


In [118]:
drop_inactive_players(1, POOL_ID)

Dropped player latrell-wrightselljr-1 from roster 53 (inactive)
Dropped player jaxson-robinson-1 from roster 53 (inactive)
Dropped player keshon-gilbert-1 from roster 55 (inactive)
Dropped 3 inactive players from draft 1


3

In [124]:
# Example usage
def maintain_rosters(draft_num, pool_id=POOL_ID):
    """Maintain roster integrity by dropping inactive players and filling gaps."""
    dropped = drop_inactive_players(draft_num, pool_id)
    added = fill_rosters_in_draft_order(draft_num, pool_id)
    print(f"Maintenance complete: {dropped} players dropped, {added} players added")


In [169]:
maintain_rosters(1, POOL_ID)

Dropped 0 inactive players from draft 1
Target roster size: 10
data=[{'ranking': 1, 'roster_id': 43, 'player_unique': 'cooper-flagg-1'}, {'ranking': 2, 'roster_id': 43, 'player_unique': 'johni-broome-1'}, {'ranking': 4, 'roster_id': 43, 'player_unique': 'kon-knueppel-1'}, {'ranking': 5, 'roster_id': 43, 'player_unique': 'tyrese-proctor-1'}, {'ranking': 6, 'roster_id': 43, 'player_unique': 'walter-claytonjr-1'}, {'ranking': 7, 'roster_id': 43, 'player_unique': 'alijah-martin-1'}, {'ranking': 8, 'roster_id': 43, 'player_unique': 'will-richard-2'}, {'ranking': 9, 'roster_id': 43, 'player_unique': 'chad-baker-mazara-1'}, {'ranking': 10, 'roster_id': 43, 'player_unique': 'lj-cryer-1'}, {'ranking': 11, 'roster_id': 43, 'player_unique': 'rj-luis-1'}, {'ranking': 12, 'roster_id': 43, 'player_unique': 'chaz-lanier-1'}, {'ranking': 13, 'roster_id': 43, 'player_unique': 'pj-haggerty-1'}, {'ranking': 14, 'roster_id': 43, 'player_unique': 'jt-toppin-1'}, {'ranking': 15, 'roster_id': 43, 'player_uni